In [ ]:
# %pip install openai python-dotenv pandas

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import json
import pandas as pd
import os

In [2]:
# OPENAI_API_KEY = "add your key here"
# print(f"OpenAI API Key: {OPENAI_API_KEY}")

load_dotenv()

True

In [3]:
client = OpenAI()

In [4]:
def generate_mcqs(context, num_questions=5):
    """
    Generate MCQs from a given context using an LLM.
    """
    prompt = f"""
    Generate {num_questions} multiple-choice questions (MCQs) based on the following context. 
    Each question should have 4 options, with one correct answer and three distractors.
    
    Context: {context}
    
    Format each question as follows:
    Q1. [Question text]
    A) [Option 1]
    B) [Option 2]
    C) [Option 3]
    D) [Option 4]
    Correct Answer: [Correct Option]
    
    Please ensure the questions are relevant, clear, and cover key aspects of the context.
    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates multiple-choice questions."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000
    )
    
    mcqs = response.choices[0].message.content
    return mcqs

In [5]:
context = """
Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods with the help of chlorophyll. 
The process involves the intake of carbon dioxide and water, which are then converted into glucose and oxygen. 
Photosynthesis occurs in the chloroplasts of plant cells, where chlorophyll captures light energy. 
This process is crucial for the production of oxygen and the removal of carbon dioxide from the atmosphere.
"""

In [6]:
mcqs = generate_mcqs(context)

In [7]:
mcqs = mcqs.split("\n\n")
mcqs

['Q1. What is the primary pigment involved in photosynthesis?\nA) Carotene  \nB) Chlorophyll  \nC) Hemoglobin  \nD) Xanthophyll  \nCorrect Answer: B) Chlorophyll',
 'Q2. In which part of the plant cell does photosynthesis take place?\nA) Mitochondria  \nB) Golgi apparatus  \nC) Nucleus  \nD) Chloroplasts  \nCorrect Answer: D) Chloroplasts',
 'Q3. What are the main inputs required for the process of photosynthesis?\nA) Oxygen and glucose  \nB) Nitrogen and phosphorus  \nC) Carbon dioxide and water  \nD) Sunlight and minerals  \nCorrect Answer: C) Carbon dioxide and water',
 'Q4. What are the primary products of photosynthesis?\nA) Carbon dioxide and glucose  \nB) Water and oxygen  \nC) Glucose and oxygen  \nD) Oxygen and nitrogen  \nCorrect Answer: C) Glucose and oxygen',
 "Q5. Why is photosynthesis considered crucial for the Earth's atmosphere?\nA) It increases carbon dioxide levels.  \nB) It produces glucose only.  \nC) It generates oxygen and reduces carbon dioxide.  \nD) It requires

In [8]:
print(mcqs[0])

Q1. What is the primary pigment involved in photosynthesis?
A) Carotene  
B) Chlorophyll  
C) Hemoglobin  
D) Xanthophyll  
Correct Answer: B) Chlorophyll


In [13]:
def evaluate_mcqs(mcqs, context):
    results = []
    for question in mcqs:
        prompt = f"""
        Evaluate the following MCQ based on the provided context.
        Return your evaluation in valid JSON format with scores (0-1) and justifications.

        Context: {context}

        MCQ: {question}

        Required JSON format:
        {{
            "question": "<question_text>",
            "evaluations": {{
                "relevance_score": <0-1>,
                "relevance_justification": "<text>",
                "clarity_score": <0-1>,
                "clarity_justification": "<text>",
                "difficulty_score": <0-1>,
                "difficulty_justification": "<text>",
                "correctness_score": <0-1>,
                "correctness_justification": "<text>"
            }}
        }}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are an MCQ evaluation assistant. Always respond with valid JSON in the exact format specified."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1000
            )

            eval_data = json.loads(response.choices[0].message.content)
            results.append({
                'Question': eval_data['question'],
                'Relevance Score': eval_data['evaluations']['relevance_score'],
                'Relevance Justification': eval_data['evaluations']['relevance_justification'],
                'Clarity Score': eval_data['evaluations']['clarity_score'],
                'Clarity Justification': eval_data['evaluations']['clarity_justification'],
                'Difficulty Score': eval_data['evaluations']['difficulty_score'],
                'Difficulty Justification': eval_data['evaluations']['difficulty_justification'],
                'Correctness Score': eval_data['evaluations']['correctness_score'],
                'Correctness Justification': eval_data['evaluations']['correctness_justification']
            })
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON for question: {question}\nError: {str(e)}")
            continue

    return pd.DataFrame(results)

In [14]:
evaluate_mcqs(mcqs, context)

,Question,Relevance Score,Relevance Justification,Clarity Score,Clarity Justification,Difficulty Score,Difficulty Justification,Correctness Score,Correctness Justification
0,What is the primary pigment involved in photos...,1,The question is directly related to the contex...,1,The question is straightforward and clear in i...,0.2,"The question is fairly easy, given the context...",1,The correct answer choice 'B) Chlorophyll' ali...
1,In which part of the plant cell does photosynt...,1,The question is highly relevant to the context...,1,The question is very clear and straightforward...,0.2,"Given the context, this question is not very d...",1,"The answer stated, chloroplasts, is correct as..."
2,What are the main inputs required for the proc...,1,The question is directly related to the contex...,1,The question is clear and easy to understand. ...,0.2,The question is relatively simple since the co...,1,The correct answer is provided as 'C) Carbon d...
3,What are the primary products of photosynthesis?,1,The question is directly related to the contex...,1,The question is clear and straightforward. It ...,0.2,The question is not very difficult as the answ...,1,The correct answer provided (glucose and oxyge...
4,Why is photosynthesis considered crucial for t...,1,The question is directly related to the provid...,1,The question is very clear and easily understa...,0.5,The difficulty level of the question is modera...,1,The correct answer as per the context is 'It g...
